In [1]:
import sys
import os
sys.path.append('/root/capsule/code/beh_ephys_analysis')
from utils.beh_functions import parseSessionID, session_dirs, get_unit_tbl, get_session_tbl
from utils.plot_utils import shiftedColorMap, template_reorder, get_gradient_colors
from utils.opto_utils import opto_metrics
from utils.ephys_functions import cross_corr_train, auto_corr_train, load_drift
import json
import matplotlib.pyplot as plt
import pandas as pd
import pickle
%matplotlib inline


In [2]:
def cross_auto_corr(session, data_type):
    bin_long = 0.05
    win_long = 2
    bin_short = 0.002
    win_short = 0.08
    session_dir = session_dirs(session)
    unit_tbl = get_unit_tbl(session, data_type) 
    if unit_tbl['LC_range_top'].unique()[0] is None:
        print(f'LC range not inferred, probably no opto units in {session}. Exiting.')
        return None
    session_tbl = get_session_tbl(session)
    
    session_qm_file = os.path.join(session_dir['processed_dir'], f'{session}_qm.json')
    with open(session_qm_file, 'r') as f:
        session_qm = json.load(f)
    rec_start = session_qm['ephys_cut'][0]
    rec_end = session_qm['ephys_cut'][1]

    opto_file = os.path.join(session_dir['opto_dir_curated'], 
                                f'{session}_opto_session.csv')
    opto_tbl = pd.read_csv(opto_file)
    opto_times = opto_tbl['time'].values

    if len(opto_tbl['pre_post'].unique()) > 1:
        rec_start = opto_tbl[opto_tbl['pre_post'] == 'pre']['time'].max()
        rec_end = opto_tbl[opto_tbl['pre_post'] == 'post']['time'].min()
    elif len(opto_tbl['pre_post'].unique()) == 1:
        rec_end = opto_tbl['time'].min()

    filter = (
        (unit_tbl['decoder_label'] != 'artifact') &
        (unit_tbl['decoder_label'] != 'noise') &
        (unit_tbl['isi_violations_ratio'] < 0.5) &
        (unit_tbl['y_loc'] >= unit_tbl['LC_range_bottom'].unique()[0] - 0.1) &
        (unit_tbl['y_loc'] <= unit_tbl['LC_range_top'].unique()[0] + 0.1)
    )

    unit_ids_focus = unit_tbl[filter]['unit_id'].to_list()
    print(f'Number of units in LC range: {len(unit_ids_focus)}')

    cross_corr_df = pd.DataFrame(columns=['unit_1', 'unit_2', 'cross_corr_long', 'cross_corr_short', 'start', 'end'])
    auto_corr_df = pd.DataFrame(columns=['unit', 'auto_corr_long', 'auto_corr_short', 'start', 'end'])
    for unit_ind_1, unit_1 in enumerate(unit_ids_focus):
        print(f'Processing unit {unit_1}')
        spike_times_1 = unit_tbl[unit_tbl['unit_id'] == unit_1]['spike_times'].values[0]
        drift_1 = load_drift(session, unit_1, data_type)
        start_unit_1 = rec_start
        end_unit_1 = rec_end
        if drift_1 is not None:
            if drift_1['ephys_cut'][0] is not None:
                start_unit_1 = max(start_unit_1, drift_1['ephys_cut'][0])
            if drift_1['ephys_cut'][1] is not None:
                end_unit_1 = min(end_unit_1, drift_1['ephys_cut'][1])
        auto_corr_long = auto_corr_train(spike_times_1, bin_long, win_long, start_unit_1, end_unit_1)
        auto_corr_short = auto_corr_train(spike_times_1, bin_short, win_short, start_unit_1, end_unit_1)
        dist_out_auto = {'unit': unit_1,
                    'auto_corr_long': auto_corr_long,
                    'auto_corr_short': auto_corr_short,
                    'start': start_unit_1,
                    'end': end_unit_1}
        auto_corr_df = pd.concat([auto_corr_df, pd.DataFrame([dist_out_auto])], ignore_index=True)
        for unit_ind_2, unit_2 in enumerate(unit_ids_focus):
            cross_corr_long = None
            cross_corr_short = None
            if unit_ind_1 < unit_ind_2:
                print(f'Processing units {unit_1} and {unit_2}')
                spike_times_2 = unit_tbl[unit_tbl['unit_id'] == unit_2]['spike_times'].values[0]
                drift_2 = load_drift(session, unit_2, data_type)
                start = start_unit_1
                end = end_unit_1
                if drift_2 is not None:
                    if drift_2['ephys_cut'][0] is not None:
                        start = max(start, drift_2['ephys_cut'][0])
                    if drift_2['ephys_cut'][1] is not None:
                        end = min(end, drift_2['ephys_cut'][1])
                
                if start >= end:
                    print(f'Skipping units {unit_1} and {unit_2} due to incompatible drift cuts')
                    continue
                cross_corr_long = cross_corr_train(spike_times_1, spike_times_2, bin_long, win_long, start, end)
                cross_corr_short = cross_corr_train(spike_times_1, spike_times_2, bin_short, win_short, start, end)

                dist_out = {'unit_1': unit_1,
                            'unit_2': unit_2,
                            'cross_corr_long': cross_corr_long,
                            'cross_corr_short': cross_corr_short,
                            'start': start,
                            'end': end}
                # Append the result to the cross_corr_df dataframe
                if cross_corr_long is not None or cross_corr_short is not None:
                    cross_corr_df = pd.concat([cross_corr_df, pd.DataFrame([dist_out])], ignore_index=True)
    # Save the results to pickle files
    cross_corr_file = os.path.join(session_dir[f'ephys_processed_dir_{data_type}'], f'{session}_{data_type}_cross_corr.pkl')
    auto_corr_file = os.path.join(session_dir[f'ephys_processed_dir_{data_type}'], f'{session}_{data_type}_auto_corr.pkl')
    with open(cross_corr_file, 'wb') as f:
        pickle.dump(cross_corr_df, f)
    with open(auto_corr_file, 'wb') as f:
        pickle.dump(auto_corr_df, f)

In [3]:
cross_auto_corr('ecephys_713854_2024-03-08_14-54-25', 'curated')

There are multiple recordings in the curated nwb directory. Picked one with units.
There are multiple recordings in the curated nwb directory. Picked one with units.
LC range not inferred, probably no opto units in ecephys_713854_2024-03-08_14-54-25. Exiting.


In [4]:
session_assets = pd.read_csv('/root/capsule/code/data_management/session_assets.csv')
session_list = session_assets['session_id']
probe_list = session_assets['probe']
probe_list = [probe for probe, session in zip(probe_list, session_list) if isinstance(session, str)]
session_list = [session for session in session_list if isinstance(session, str)]    
from joblib import Parallel, delayed
data_type = 'curated'
def process(session, data_type): 
    print(f'Starting {session}')
    session_dir = session_dirs(session)
    # if os.path.exists(os.path.join(session_dir['beh_fig_dir'], f'{session}.nwb')):
    print(session_dir[f'curated_dir_{data_type}'])
    if session_dir[f'curated_dir_{data_type}'] is not None:
        try:
            # plot_ephys_probe(session, data_type=data_type, probe=probe) 
            cross_auto_corr(session, data_type)
            plt.close('all')
            print(f'Finished {session}')
        except:
            print(f'Error processing {session}')
            plt.close('all')
    else: 
        print(f'No curated data found for {session}') 
    # elif session\_dir['curated_dir_raw'] is not None:
    #     data_type = 'raw' 
    #     opto_tagging_df_sess = opto_plotting_session(session, data_type, target, resp_thresh=resp_thresh, lat_thresh=lat_thresh, target_unit_ids= None, plot = True, save=True)
Parallel(n_jobs=-8)(
    delayed(process)(session, data_type) 
    for session in session_list
)

# process('behavior_754897_2025-03-13_11-20-42', data_type)
# for session, probe in zip(session_list, probe_list):
#     process(session, data_type, probe)
#     plt.close('all')

Starting ecephys_713854_2024-03-05_12-01-40
/root/capsule/data/ecephys_713854_2024-03-05_12-01-40_sorted_curated/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
Starting ecephys_717120_2024-03-07_12-12-02
/root/capsule/data/ecephys_717120_2024-03-07_12-12-02_sorted_curated/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
Starting ecephys_713854_2024-03-05_13-31-20
/root/capsule/data/ecephys_713854_2024-03-05_13-31-20_sorted_curated/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
Starting ecephys_713854_2024-03-05_13-01-09
/root/capsule/data/ecephys_713854_2024-03-05_13-01-09_sorted_curated/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
Starting ecephys_713854_2024-03-08_15-43-01
/root/capsule/data/ecephys_713854_2024-03-08_15-43-01_sorted_curated/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
Starting ecephys_713854_2024-03-08_16-20-33
/root/capsule/data/ecephys_713854_20

/tmp/ipykernel_16383/1956190056.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/tmp/ipykernel_16383/1956190056.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/tmp/ipykernel_16383/1956190056.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/

Processing units 0 and 3
Processing units 3 and 4
Processing units 4 and 5
Starting ecephys_717120_2024-03-06_12-23-53
/root/capsule/data/ecephys_717120_2024-03-06_12-23-53_sorted_curated/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
Number of units in LC range: 124
Processing unit 22.0


/tmp/ipykernel_16383/1956190056.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/tmp/ipykernel_16383/1956190056.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/tmp/ipykernel_16383/1956190056.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/

Processing units 21 and 24
LC range not inferred, probably no opto units in ecephys_717120_2024-03-06_12-23-53. Exiting.
Starting ecephys_713854_2024-03-08_14-54-25
There are multiple recordings in the curated nwb directory. Picked one with units.
Finished ecephys_717120_2024-03-06_12-23-53
/root/capsule/data/ecephys_713854_2024-03-08_14-54-25_sorted_curated/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
There are multiple recordings in the curated nwb directory. Picked one with units.
There are multiple recordings in the curated nwb directory. Picked one with units.
Processing units 6 and 12
Processing units 3 and 5
Processing units 52 and 54
Processing units 0 and 4
Processing units 3 and 7
Starting ecephys_684930_2023-09-28_11-45-27
/root/capsule/data/ecephys_684930_2023-09-28_11-45-27_sorted_curated/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA-AP_recording1


...

/tmp/ipykernel_16383/1956190056.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


Processing units 0.0 and 2.0
Processing units 22.0 and 68.0
Processing units 6 and 56
Processing units 3 and 19
Processing units 21 and 133
Processing units 7 and 24
Processing units 5.0 and 75.0
Processing units 13.0 and 49.0
Processing units 7 and 126
Processing units 3 and 22
Processing units 5 and 176
Processing units 21 and 134
Processing units 22.0 and 69.0
Processing units 6 and 57
Processing units 0.0 and 4.0
Processing units 5.0 and 78.0
Skipping units 0.0 and 4.0 due to incompatible drift cuts
Processing units 0.0 and 6.0


/tmp/ipykernel_16383/1956190056.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


Skipping units 0.0 and 6.0 due to incompatible drift cuts
Processing units 0.0 and 7.0
Processing units 13.0 and 50.0
Processing units 7 and 165
Processing units 5 and 182
Processing units 3 and 24
Processing units 6 and 58
Processing units 21 and 137
Processing units 5.0 and 79.0
Processing units 7 and 166
Processing units 0.0 and 18.0
Processing units 22.0 and 70.0
Processing units 13.0 and 51.0
Processing units 5 and 184
Processing units 3 and 27
Processing units 6 and 59
Processing units 7 and 172
Processing units 21 and 138
Processing units 5.0 and 80.0
Processing units 5 and 219
Processing units 3 and 28
Processing units 22.0 and 72.0
Processing units 13.0 and 52.0
Processing units 6 and 60
Processing units 7 and 173
Processing units 0.0 and 22.0
Processing units 21 and 145
Processing units 5.0 and 83.0
Processing units 3 and 29
Processing units 6 and 64
Processing units 5 and 232
Processing unit 10
Processing units 21 and 146
Processing units 22.0 and 75.0
Processing units 0.0 a

KeyboardInterrupt: 

['ecephys_713854_2024-03-05_12-01-40',
 'ecephys_713854_2024-03-05_13-01-09',
 'ecephys_713854_2024-03-05_13-31-20',
 'ecephys_717120_2024-03-06_12-23-53',
 'ecephys_717120_2024-03-06_12-54-27',
 'ecephys_717120_2024-03-07_12-12-02',
 'ecephys_713854_2024-03-08_14-54-25',
 'ecephys_713854_2024-03-08_15-43-01',
 'ecephys_713854_2024-03-08_16-20-33',
 'ecephys_713854_2024-03-08_17-15-58',
 'ecephys_684930_2023-09-27_10-04-04',
 'ecephys_684930_2023-09-28_11-45-27',
 'ecephys_684930_2023-09-28_12-44-15',
 'ecephys_687697_2023-09-15_11-30-06',
 'ecephys_687697_2023-09-15_12-36-06',
 'ecephys_691893_2023-10-05_12-46-57',
 'ecephys_691893_2023-10-06_13-48-18',
 'behavior_716325_2024-05-29_10-33-32',
 'behavior_716325_2024-05-30_11-33-46',
 'behavior_716325_2024-05-31_10-31-14',
 'behavior_716325_2024-06-01_09-25-37',
 'behavior_717121_2024-06-11_10-23-31',
 'behavior_717121_2024-06-13_12-21-20',
 'behavior_717121_2024-06-14_10-23-49',
 'behavior_717121_2024-06-15_10-00-58',
 'behavior_717121

In [23]:
cross_corr_df

,unit_1,unit_2,cross_corr_long,cross_corr_short,start,end
0,10,11,"([0.10200926919638711, 0.04362907474724333, 0....","([0.0036463771130829893, 0.008541877187731667,...",7.165321e+06,7.171568e+06
1,10,12,"([0.07899574598134228, 0.04555429368121589, 0....","([0.009465412686014389, 0.005973386332210205, ...",7.165321e+06,7.171568e+06
2,10,15,"([0.0527512776581984, 0.043061203472371384, 0....","([0.004471735748298048, 0.004735447002790088, ...",7.165321e+06,7.171568e+06
3,10,18,"([0.017524276074430887, -0.011870131140818849,...","([0.005815145606901504, 0.003577636879219868, ...",7.165321e+06,7.171568e+06
4,10,23,"([0.04341201869179076, 0.041732545063786194, 0...","([0.004937083740363691, 0.001273885942614803, ...",7.165321e+06,7.171568e+06
5,10,24,"([0.04589335236936446, 0.03281382579775501, 0....","([0.004746920685482899, 0.0033847179703455055,...",7.165321e+06,7.171568e+06
6,10,25,"([0.11744807261371962, 0.08945177046470547, 0....","([0.007906761137817098, 0.007292462947555057, ...",7.165321e+06,7.171568e+06
